In [ ]:
#!/usr/bin/env python3
"""
activation_probe_detailed.py — Phase 3.1–3.2 (Final)
Captures both full token-wise and mean-pooled activations
from all transformer layers of Qwen2.5-3B-Instruct.

Output structure:
activations/
 ├─ prompt01/
 │   ├─ layer00_full.npy
 │   ├─ layer00_pooled.npy
 │   ├─ ...
 │   └─ layer35_pooled.npy
 ├─ prompt02/
 │   └─ ...
"""
import os
import json
import torch
import numpy as np
from datetime import datetime
from transformers import AutoTokenizer, AutoModelForCausalLM


# ---------------------------------------------------------------------
# 1. Model Loading
# ---------------------------------------------------------------------
def load_model(model_name="Qwen/Qwen2.5-3B-Instruct"):
    device = "cuda" if torch.cuda.is_available() else "cpu"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
        device_map="auto"
    )
    model.eval()
    n_layers = len(model.model.layers)
    print(f"[INFO] Loaded {model_name} on {device} with {n_layers} transformer layers.")
    return model, tokenizer, device, n_layers


# ---------------------------------------------------------------------
# 2. Hook registration (safe)
# ---------------------------------------------------------------------
def register_hooks(model, store):
    """Attach forward hooks that safely copy activations to CPU."""
    handles = []
    for idx, layer in enumerate(model.model.layers):
        def hook_fn(module, inp, out, layer_idx=idx):
            store[layer_idx] = out[0].detach().cpu()
        handles.append(layer.register_forward_hook(hook_fn))
    return handles


# ---------------------------------------------------------------------
# 3. Activation Capture
# ---------------------------------------------------------------------
def capture_activations(model, tokenizer, device, prompts, save_dir="activations"):
    os.makedirs(save_dir, exist_ok=True)
    store = {}
    hooks = register_hooks(model, store)

    with torch.no_grad():
        for i, prompt in enumerate(prompts, start=1):
            store.clear()
            inputs = tokenizer(prompt, return_tensors="pt").to(device)
            _ = model(**inputs)

            prompt_dir = os.path.join(save_dir, f"prompt{i:02d}")
            os.makedirs(prompt_dir, exist_ok=True)

            for layer_idx, tensor in store.items():
                # Save full token activations: (seq_len, hidden_dim)
                full = tensor.squeeze(0).cpu().numpy()
                np.save(f"{prompt_dir}/layer{layer_idx:02d}_full.npy", full)

                # Save mean-pooled activations: (hidden_dim,)
                pooled = full.mean(axis=0)
                np.save(f"{prompt_dir}/layer{layer_idx:02d}_pooled.npy", pooled)

            print(f"[INFO] Saved activations for prompt {i}: "
                  f"{len(store)} layers × 2 versions (full & pooled)")

    # Remove hooks after all prompts processed
    for h in hooks:
        h.remove()

    print(f"[INFO] Activation extraction complete → {save_dir}/")


# ---------------------------------------------------------------------
# 4. Main Entry
# ---------------------------------------------------------------------
def main():
    # Load latest recommender JSON log
    log_dir = "logs"
    log_files = sorted([
        f for f in os.listdir(log_dir)
        if f.startswith("recommender_") and f.endswith(".json")
    ])
    if not log_files:
        raise FileNotFoundError("No recommender_*.json log found.")
    latest_log = os.path.join(log_dir, log_files[-1])

    with open(latest_log, "r", encoding="utf-8") as f:
        data = json.load(f)
    prompts = [r["question"] for r in data["records"]]

    model, tokenizer, device, n_layers = load_model()
    capture_activations(model, tokenizer, device, prompts)


if __name__ == "__main__":
    main()


/home/rameyjm7/workspace/TML/lpu/llm-preference-unlearning/lpu-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
`torch_dtype` is deprecated! Use `dtype` instead!
Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.08it/s]


[INFO] Loaded Qwen/Qwen2.5-3B-Instruct on cuda with 36 transformer layers.
[INFO] Saved activations for prompt 1: 36 layers × 2 versions (full & pooled)
[INFO] Saved activations for prompt 2: 36 layers × 2 versions (full & pooled)
[INFO] Saved activations for prompt 3: 36 layers × 2 versions (full & pooled)
[INFO] Saved activations for prompt 4: 36 layers × 2 versions (full & pooled)
[INFO] Saved activations for prompt 5: 36 layers × 2 versions (full & pooled)
[INFO] Activation extraction complete → activations/


: 